# Preprocessing Data 

### 1-  Merge flights  Data with origin JFK airport.

In [1]:
## Import basic packages we will use 
import pandas as pd
import datetime
from datetime import datetime
import numpy as np
import glob
import os
from datetime import datetime, timedelta



In the next code chuck:
We merge all Airlines Data from 2018 to 2022 of flights from JFK airports in a CSV file "JFK_merged_18_22.csv". We only consider flights with the origin JFK-airport.

In [ ]:
# full path of a folder of the Airlines Data from 2018 to 2022  
path = "Path/..../folder_name=JFK-2018-2022/*.csv"

# get filesnames
filenames=glob.glob(path)
#   List of file names
list_of_dfs = [pd.read_csv(filename) for filename in filenames]

# merge all data in a single dataframe.
all_df=pd.concat(list_of_dfs)

## Flights Origin JFK.


JFK_df=all_df[all_df.Origin=='JFK']


## Save ALL DATA in a single cvs file:  JFK_merged_18_22.csv
JFK_df.to_csv(path_or_buf ="path/..../folder_name=JFK-2018-2022/JFK_merged_18_22.csv" , index = False)


### 2- Preprocessin Airlines Data Data 

In [2]:
# Loading data 
JFK_df = pd.read_csv("path/....../JFK_merged_18_22.csv").copy() 


In [3]:
JFK_df.head()

,Unnamed: 0.4,Unnamed: 0.1,Unnamed: 0,DayOfWeek,FL_Date,Carrier,FL_Num,Origin,Dest,Scheduled_Dep_Time,...,Estimated_Elapsed_Time,Actual_Elapsed_Time,Distance,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,Unnamed: 0.3,Unnamed: 0.2
0,0,82156.0,82156.0,2,2022-05-31,UA,523,JFK,SFO,1730,...,385.0,384.0,2586.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,82159.0,82159.0,2,2022-05-31,UA,521,JFK,SFO,630,...,380.0,371.0,2586.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,82162.0,82162.0,2,2022-05-31,UA,517,JFK,LAX,1835,...,380.0,359.0,2475.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,82164.0,82164.0,2,2022-05-31,UA,515,JFK,LAX,1000,...,381.0,341.0,2475.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,83934.0,83934.0,1,2022-05-30,UA,523,JFK,SFO,1730,...,385.0,NaN,2586.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
print("Length of JFK_dF with origin JFK in 2018 to 2022 ",len(JFK_df))
print()
print()
print("Columns of JFK_dF with origin JFK in 2018 to 2022     : ",    list(JFK_df.columns))

In [4]:
# drop added Unnamed columns during the merge 
JFK_df= JFK_df.drop(['Unnamed: 0.4', 'Unnamed: 0.1', 'Unnamed: 0', 'Unnamed: 0.3', 'Unnamed: 0.2'], axis = 1)


In [5]:
# data info 
JFK_df.isnull().sum()

DayOfWeek                      0
FL_Date                        0
Carrier                        0
FL_Num                         0
Origin                         0
Dest                           0
Scheduled_Dep_Time             0
Actual_Dep_Time            14614
DepDelay                   14749
DepartureDelayGroups       14749
TaxiOut                    15244
Scheduled_Arr_Time             0
Actual_Arr_Time            15410
ArrDelay                   16967
ArrivalDelayGroups         16967
Cancelled                      0
CancellationCode          619875
Diverted                       0
Estimated_Elapsed_Time         0
Actual_Elapsed_Time        16916
Distance                       0
CarrierDelay              516461
WeatherDelay              516461
NASDelay                  516461
SecurityDelay             516461
LateAircraftDelay         516461
dtype: int64

### Converting time columns into useful datetime objects 
We will only consider data needed for our model and it is avaible before the flight departure time

In [6]:
# drop rows associated to cancelled flights 'Actual_Dep_Time'==NaN i.e., cancelled flights 

JFK_df=JFK_df[JFK_df['Actual_Dep_Time'].notnull()] # drop cancelled flights
JFK_df=JFK_df[JFK_df['Actual_Arr_Time'].notnull()] # drop cancelled flights


In [7]:
# drop not needed columns
JFK_df= JFK_df.drop(['FL_Num', 'DepartureDelayGroups', 'TaxiOut'], axis = 1)
JFK_df= JFK_df.drop(['Scheduled_Arr_Time', 'Actual_Arr_Time', 'ArrDelay', 'ArrivalDelayGroups'], axis = 1)
JFK_df= JFK_df.drop(['Cancelled', 'CancellationCode', 'Diverted', 'Estimated_Elapsed_Time', 'Actual_Elapsed_Time'],
                    axis = 1)
 

In [ ]:
## Keeping only the usable features
print("Columns of JFK_dF with origin JFK in 2018 to 2022     : ",    list(JFK_df.columns))

In [8]:
def make_datetime(df,colname):
    df[colname]=pd.to_numeric(df[colname])
    minute=(df[colname]%100).values
    hr=(df[colname]//100).values
    time_col=[]
  
    for i in range(len(minute)):
        
        time_col.append(df.FL_Date.iloc[i] +timedelta(hours=int(hr[i]),minutes=int(minute[i])))
    return time_col


JFK_df['FL_Date'] = pd.to_datetime(JFK_df.FL_Date)

JFK_df['Actual_Dep_Time']=make_datetime(JFK_df,'Actual_Dep_Time')
JFK_df['Scheduled_Dep_Time']=make_datetime(JFK_df,'Scheduled_Dep_Time')



JFK_df['Year']=pd.DatetimeIndex(JFK_df['FL_Date']).year
JFK_df['Month']=pd.DatetimeIndex(JFK_df['FL_Date']).month
JFK_df['Hour']=pd.DatetimeIndex(JFK_df['Scheduled_Dep_Time']).hour

# create a new column, we consider two Status 

JFK_df.loc[JFK_df['DepDelay'] <= 15, 'Status'] = 0
JFK_df.loc[JFK_df['DepDelay'] > 15, 'Status'] = 1


In [ ]:
# Save JFK_df  data frame into csv file;
#JFK_df.to_csv(path_or_buf = "Path/..../folder_name=JFK-2018-2022/DataPrep_JFK_merged_18_22.csv" , index = False)

# Loading JFK_df_sort data 
JFK_df = pd.read_csv("Path/..../folder_name=JFK-2018-2022/DataPrep_JFK_merged_18_22.csv").copy() 


Next, we merge all weather Data from 2018 to 2022 for new york city, JFK airport, in a single CSV file "Weather_merged_18_22.csv". 

In [ ]:
# Full path of the Weather Data folder, from 2018 to 2022 

path = "Path/..../folder_name=Weather-2018-2022/*.csv"


# get filesnames
filenames=glob.glob(path)
#   List of file names
list_of_dfs = [pd.read_csv(filename) for filename in filenames]

# merge all data in a single dataframe.
Weather_df=pd.concat(list_of_dfs)

## Save ALL DATA in a single cvs file:  Weather_Data_18_22.csv
Weather_df.to_csv(path_or_buf ="Path/..../folder_name=Weather-2018-2022/Weather_merged_18_22.csv" , index = False)


In [9]:
# Loading weather  data 
Weather_df = pd.read_csv('Path/..../folder_name=Weather-2018-2022/Weather_merged_18_22.csv').copy() 
Weather_df


,Day,time-1,Temperature_F,Dew_Point_F,Humidity_%,Wind,Wind_Speed_mph,Wind_Gust_mph,Pressure_in,Precip_in,Condition
0,2018-04-01,11:51 PM,41,37,86,S,16,0,30.12,0.0,Partly Cloudy
1,2018-04-01,12:51 AM,42,37,82,S,21,0,30.09,0.0,Mostly Cloudy / Windy
2,2018-04-01,1:51 AM,43,38,82,SSW,20,0,30.06,0.0,Mostly Cloudy
3,2018-04-01,2:51 AM,43,39,86,SSW,16,0,30.03,0.0,Cloudy
4,2018-04-01,3:51 AM,43,40,89,SSW,13,0,30.01,0.0,Cloudy
...,...,...,...,...,...,...,...,...,...,...,...
55905,2022-05-31,6:51 PM,75,66,73,SSW,6,0,29.88,0.0,Mostly Cloudy
55906,2022-05-31,7:51 PM,74,65,73,SE,3,0,29.89,0.0,Mostly Cloudy
55907,2022-05-31,8:51 PM,75,66,73,E,10,0,29.92,0.0,Mostly Cloudy
55908,2022-05-31,9:51 PM,67,62,84,E,10,0,29.93,0.0,Mostly Cloudy


In [10]:
## Converting time columns into datetime objects
Weather_df.Day = pd.to_datetime(Weather_df.Day)


def make_datetime(df):
    hour= pd.to_datetime(df["time-1"],format = '%I:%M %p').dt.hour
    minute=pd.to_datetime(df["time-1"],format = '%I:%M %p').dt.minute
    time_col=[]
  
    for i in range(len(minute)):
        time_col.append(df.Day.iloc[i] +timedelta(hours=int(hour[i]),minutes=int(minute[i])))
    return time_col

## Converting time columns into useful (merge process) datetime objects

Weather_df['Full_time']=make_datetime(Weather_df)



Weather_df.head()


,Day,time-1,Temperature_F,Dew_Point_F,Humidity_%,Wind,Wind_Speed_mph,Wind_Gust_mph,Pressure_in,Precip_in,Condition,Full_time
0,2018-04-01,11:51 PM,41,37,86,S,16,0,30.12,0.0,Partly Cloudy,2018-04-01 23:51:00
1,2018-04-01,12:51 AM,42,37,82,S,21,0,30.09,0.0,Mostly Cloudy / Windy,2018-04-01 00:51:00
2,2018-04-01,1:51 AM,43,38,82,SSW,20,0,30.06,0.0,Mostly Cloudy,2018-04-01 01:51:00
3,2018-04-01,2:51 AM,43,39,86,SSW,16,0,30.03,0.0,Cloudy,2018-04-01 02:51:00
4,2018-04-01,3:51 AM,43,40,89,SSW,13,0,30.01,0.0,Cloudy,2018-04-01 03:51:00


In [ ]:
## Save Preprocessing weather Data Weather_df into new Weather_prep_merged_18_22.csv file  


Weather_df.to_csv(path_or_buf =" ......./Weather-2018-2022/Weather_prep_merged_18_22.csv" , index = False)

Weather_df = pd.read_csv('...../Weather_prep_merged_18_22.csv').copy() 
Weather_df




### In order to merge JFK_df with Weather_df_data we sort and set index 'Actual_Dep_Time'

In [11]:
# sort and set merge index for JFK_df

JFK_df_sort=JFK_df.sort_values(by=['Actual_Dep_Time'])
JFK_df_sort= JFK_df_sort.set_index('Actual_Dep_Time')
JFK_df_sort.head()

,DayOfWeek,FL_Date,Carrier,Origin,Dest,Scheduled_Dep_Time,DepDelay,Distance,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,Year,Month,Hour,Status
Actual_Dep_Time,,,,,,,,,,,,,,,,,
2018-01-01 00:12:00,1,2018-01-01,B6,JFK,SYR,2018-01-01 22:52:00,80.0,209.0,10.0,0.0,0.0,0.0,59.0,2018,1,22,1.0
2018-01-01 02:38:00,1,2018-01-01,B6,JFK,BOS,2018-01-01 22:46:00,232.0,187.0,56.0,0.0,0.0,0.0,162.0,2018,1,22,1.0
2018-01-01 05:21:00,1,2018-01-01,B6,JFK,PBI,2018-01-01 05:30:00,-9.0,1028.0,NaN,NaN,NaN,NaN,NaN,2018,1,5,0.0
2018-01-01 05:42:00,1,2018-01-01,B6,JFK,FLL,2018-01-01 05:45:00,-3.0,1069.0,NaN,NaN,NaN,NaN,NaN,2018,1,5,0.0
2018-01-01 05:54:00,1,2018-01-01,AA,JFK,PHX,2018-01-01 06:00:00,-6.0,2153.0,NaN,NaN,NaN,NaN,NaN,2018,1,6,0.0


In [12]:

# sort and set merge index for Weather_df
Weather_df_sort= Weather_df.set_index('Full_time').sort_index()
Weather_df_sort=Weather_df_sort.drop(["Day","time-1"],axis=1)
Weather_df_sort.sort_index()
Weather_df_sort.head()

,Temperature_F,Dew_Point_F,Humidity_%,Wind,Wind_Speed_mph,Wind_Gust_mph,Pressure_in,Precip_in,Condition
Full_time,,,,,,,,,
2018-01-01 00:51:00,9,-3,59,NNW,18,0,30.33,0.0,Fair
2018-01-01 00:51:00,9,-3,59,NNW,18,0,30.33,0.0,Fair
2018-01-01 01:51:00,8,-3,61,NW,14,0,30.34,0.0,Fair
2018-01-01 01:51:00,8,-3,61,NW,14,0,30.34,0.0,Fair
2018-01-01 02:51:00,8,-4,58,NNW,14,0,30.34,0.0,Fair


In [16]:

# rename column to merge 
JFK_df_sort=JFK_df_sort.rename(columns={"Actual_Dep_Time": "TIME"})
Weather_df_sort=Weather_df_sort.rename(columns={"Full_time": "TIME"})
tol = pd.Timedelta('1 hour')
merged_df = pd.merge_asof(JFK_df_sort, Weather_df_sort,
                       right_index=True,left_index=True, direction='nearest', tolerance=tol)
                
display(merged_df)                     



,DayOfWeek,FL_Date,Carrier,Origin,Dest,Scheduled_Dep_Time,DepDelay,Distance,CarrierDelay,WeatherDelay,...,Status,Temperature_F,Dew_Point_F,Humidity_%,Wind,Wind_Speed_mph,Wind_Gust_mph,Pressure_in,Precip_in,Condition
Actual_Dep_Time,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:12:00,1,2018-01-01,B6,JFK,SYR,2018-01-01 22:52:00,80.0,209.0,10.0,0.0,...,1.0,9.0,-3.0,59.0,NNW,18.0,0.0,30.33,0.0,Fair
2018-01-01 02:38:00,1,2018-01-01,B6,JFK,BOS,2018-01-01 22:46:00,232.0,187.0,56.0,0.0,...,1.0,8.0,-4.0,58.0,NNW,14.0,0.0,30.34,0.0,Fair
2018-01-01 05:21:00,1,2018-01-01,B6,JFK,PBI,2018-01-01 05:30:00,-9.0,1028.0,NaN,NaN,...,0.0,7.0,-5.0,58.0,NW,17.0,0.0,30.34,0.0,Fair
2018-01-01 05:42:00,1,2018-01-01,B6,JFK,FLL,2018-01-01 05:45:00,-3.0,1069.0,NaN,NaN,...,0.0,7.0,-5.0,58.0,NW,16.0,0.0,30.35,0.0,Fair
2018-01-01 05:54:00,1,2018-01-01,AA,JFK,PHX,2018-01-01 06:00:00,-6.0,2153.0,NaN,NaN,...,0.0,7.0,-5.0,58.0,NW,16.0,0.0,30.35,0.0,Fair
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31 22:50:00,3,2022-08-31,9E,JFK,BWI,2022-08-31 22:50:00,0.0,184.0,NaN,NaN,...,0.0,77.0,51.0,40.0,W,9.0,0.0,29.85,0.0,Fair
2022-08-31 22:55:00,3,2022-08-31,B6,JFK,PSE,2022-08-31 22:59:00,-4.0,1617.0,NaN,NaN,...,0.0,77.0,51.0,40.0,W,9.0,0.0,29.85,0.0,Fair
2022-08-31 23:02:00,3,2022-08-31,B6,JFK,BOS,2022-08-31 22:55:00,7.0,187.0,NaN,NaN,...,0.0,77.0,51.0,40.0,W,9.0,0.0,29.85,0.0,Fair


In [24]:
merged_df.to_csv(path_or_buf ="path/.../merged_JKF_Weather_prep_df_18_22.csv" , index = True)


In [25]:
merged_prep_df = pd.read_csv('path/.../merged_JKF_Weather_prep_df_18_22.csv').copy() 
merged_prep_df

,Actual_Dep_Time,DayOfWeek,FL_Date,Carrier,Origin,Dest,Scheduled_Dep_Time,DepDelay,Distance,CarrierDelay,...,Status,Temperature_F,Dew_Point_F,Humidity_%,Wind,Wind_Speed_mph,Wind_Gust_mph,Pressure_in,Precip_in,Condition
0,2018-01-01 00:12:00,1,2018-01-01,B6,JFK,SYR,2018-01-01 22:52:00,80.0,209.0,10.0,...,1.0,9.0,-3.0,59.0,NNW,18.0,0.0,30.33,0.0,Fair
1,2018-01-01 02:38:00,1,2018-01-01,B6,JFK,BOS,2018-01-01 22:46:00,232.0,187.0,56.0,...,1.0,8.0,-4.0,58.0,NNW,14.0,0.0,30.34,0.0,Fair
2,2018-01-01 05:21:00,1,2018-01-01,B6,JFK,PBI,2018-01-01 05:30:00,-9.0,1028.0,NaN,...,0.0,7.0,-5.0,58.0,NW,17.0,0.0,30.34,0.0,Fair
3,2018-01-01 05:42:00,1,2018-01-01,B6,JFK,FLL,2018-01-01 05:45:00,-3.0,1069.0,NaN,...,0.0,7.0,-5.0,58.0,NW,16.0,0.0,30.35,0.0,Fair
4,2018-01-01 05:54:00,1,2018-01-01,AA,JFK,PHX,2018-01-01 06:00:00,-6.0,2153.0,NaN,...,0.0,7.0,-5.0,58.0,NW,16.0,0.0,30.35,0.0,Fair
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619750,2022-08-31 22:50:00,3,2022-08-31,9E,JFK,BWI,2022-08-31 22:50:00,0.0,184.0,NaN,...,0.0,77.0,51.0,40.0,W,9.0,0.0,29.85,0.0,Fair
619751,2022-08-31 22:55:00,3,2022-08-31,B6,JFK,PSE,2022-08-31 22:59:00,-4.0,1617.0,NaN,...,0.0,77.0,51.0,40.0,W,9.0,0.0,29.85,0.0,Fair
619752,2022-08-31 23:02:00,3,2022-08-31,B6,JFK,BOS,2022-08-31 22:55:00,7.0,187.0,NaN,...,0.0,77.0,51.0,40.0,W,9.0,0.0,29.85,0.0,Fair
619753,2022-08-31 23:31:00,3,2022-08-31,B6,JFK,LAX,2022-08-31 22:45:00,46.0,2475.0,15.0,...,1.0,74.0,71.0,91.0,WNW,3.0,0.0,29.75,0.0,Cloudy
